In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold,cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_excel("D:\\GaTech\\BIOL 8901\\Metabolomic Project\\metabolomic_data.xlsx", sheet_name="1-clean data")
data.rename(columns = {'Unnamed: 0': 'ID'}, inplace = True)
sample = pd.read_csv("D:\\GaTech\\BIOL 8901\\sample_info.csv")
sample.rename(columns = {'CCLE_Name':'ID'}, inplace=True)
merged_data = data.merge(sample, on='ID')
drug = pd.read_csv('D:\\GaTech\\BIOL 8901\\sanger-dose-response.csv')
drug.rename(columns={'ARXSPAN_ID':'DepMap_ID'}, inplace = True)
working_data = merged_data.merge(drug, on='DepMap_ID')

C:\Users\nilav\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
working_data.fillna(0, inplace=True)

In [4]:
X = working_data.select_dtypes('float64')

In [6]:
X_min = X.min()
X_max = X.max()
X_range = (X_max-X_min)
X_scaled = (X-X_min)/(X_range)
X_scaled.head(1)

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,IC50_PUBLISHED,AUC_PUBLISHED,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2
0,0.3953,0.621262,0.546764,0.459783,0.417356,0.380599,0.587679,0.438061,0.405262,0.587155,...,0.000006,0.892457,5.145282e-07,2.412979e-306,0.84005,1.0,0.90019,0.516014,1.779677e-07,1.0


In [12]:
drug_mean = X_scaled['IC50_PUBLISHED'].mean()

In [26]:
ic, labels = X_scaled['IC50_PUBLISHED'], []
for i in range(len(ic)):
    if ic[i] > drug_mean:
        labels.append(1.0)
    elif ic[i] < drug_mean:
        labels.append(0.0)
        
X_scaled['labels'] = labels
y = X_scaled['labels']

In [28]:
y.value_counts()

0.0    209880
1.0     42006
Name: labels, dtype: int64

In [33]:
kf3 = KFold(n_splits=3, shuffle=False)
# loo = LeaveOneOut()
# loo.get_n_splits(X_scaled)
i=1 
model = svm.SVC(kernel='linear', C=1)
for train_index, test_index in kf3.split(X_scaled):
    X_train, X_test = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print(f"Current fold iteration: {i}")
    model.fit(X_train, y_train)
    print(pd.DataFrame({"Accurancy on Train":[accuracy_score(y_train, model.predict(X_train))],
    "Accurancy on Test":[accuracy_score(y_test, model.predict(X_test))]}))
    i+=1

Current fold iteration: 1
   Accurancy on Train  Accurancy on Test
0                 1.0                1.0
Current fold iteration: 2
   Accurancy on Train  Accurancy on Test
0                 1.0                1.0
Current fold iteration: 3
   Accurancy on Train  Accurancy on Test
0                 1.0                1.0
